# Capstone project, Part 2
for Zoomcamp, Ahmed Yahia Kallel

A continuation of the notebook, with focus on the neural networks. 

So far, these are the best output
* GB-lr003nest1000
* training accuracy: 0.494942
* validation accuracy: 0.544177

# Train.py (part of)

In [2]:

# libarires
import numpy as np
import pandas as pd

#splitter
from sklearn.model_selection import train_test_split

#pipeline
from sklearn.pipeline import Pipeline

#transformers
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler


#model
from sklearn.ensemble import GradientBoostingRegressor

#save pickle
import pickle

#read database
df = pd.read_csv("Property Prices in Tunisia.csv") #load file

#translation
df.replace(df["category"].unique(), ['Land and Farms', 'Apartments', 'Holiday rentals',
       'Shops, Businesses and Industrial Premises', 'Houses and Villas',
       'Flatshare', 'Offices and Trays'],inplace=True)

df.replace(df["type"].unique(), ['For sale', 'For rent'],inplace=True)

df.replace("Autres villes","Others", inplace=True)

del df["price"] 


# data splitting

#split according to random state = 1
df_trainval, df_test = train_test_split(df, test_size=0.2, random_state=1) #split train+val [80%], test [20%]
df_train, df_val = train_test_split(df_trainval, test_size=0.25, random_state=1) #split train[60%] val [20%]

#reset, drop index
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

y_test = df_test["log_price"].values
y_train = df_train["log_price"].values
y_val = df_val["log_price"].values

del df_test["log_price"]
del df_train["log_price"]
del df_val["log_price"]



# feature types
feat_cat = ['category','type','city','region']
feat_num = ['room_count','bathroom_count','size']

## transform

In [3]:
transformations = [
    ('Scal_num', RobustScaler() , feat_num),
    ('categorical', OneHotEncoder(dtype=np.int32,handle_unknown = 'ignore'), feat_cat)
]
transf = ColumnTransformer(transformations, remainder='drop')



# Special NN Libs

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from keras.wrappers.scikit_learn import KerasRegressor

### No bias, no extra layer

In [5]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(273,use_bias=False))
    model.add(layers.Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
kr = KerasRegressor(build_fn=build_model )

C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/1675294560.py:7: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model )


In [6]:
pipeline = Pipeline([
    ('transformer', transf),
    ('nn', kr)
])
pipeline.fit(df_train, y_train)

C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 273), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


239/239 [==============================] - 1s 925us/step - loss: 2.0109


Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('Scal_num', RobustScaler(),
                                                  ['room_count',
                                                   'bathroom_count', 'size']),
                                                 ('categorical',
                                                  OneHotEncoder(dtype=<class 'numpy.int32'>,
                                                                handle_unknown='ignore'),
                                                  ['category', 'type', 'city',
                                                   'region'])])),
                ('nn',
                 <keras.wrappers.scikit_learn.KerasRegressor object at 0x00000167378EB340>)])

In [7]:


def rms(x):
    return np.sqrt(np.mean(x**2))
print('Training loss: %2.3f' % rms(y_train- pipeline.predict(df_train)))
print('Validation loss: %2.3f' % rms(y_val- pipeline.predict(df_val)))

Training loss: 0.548
Validation loss: 0.566


### With bias

In [8]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(273,use_bias=True))
    model.add(layers.Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
kr = KerasRegressor(build_fn=build_model, epochs=10, batch_size=100 )

pipeline = Pipeline([
    ('transformer', transf),
    ('nn', kr)
])
pipeline.fit(df_train, y_train)

def rms(x):
    return np.sqrt(np.mean(x**2))
print('Training loss: %2.3f' % rms(y_train- pipeline.predict(df_train)))
print('Validation loss: %2.3f' % rms(y_val- pipeline.predict(df_val)))

Epoch 1/10


C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/2286808003.py:7: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model, epochs=10, batch_size=100 )
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_1/dense_2/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_1/dense_2/embedding_lookup_sparse/Reshape:0", shape=(None, 273), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_1/dense_2/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


77/77 [==============================] - 0s 2ms/step - loss: 4.9103
Epoch 2/10
77/77 [==============================] - 0s 1ms/step - loss: 0.4004
Epoch 3/10
77/77 [==============================] - 0s 2ms/step - loss: 0.2994
Epoch 4/10
77/77 [==============================] - 0s 1ms/step - loss: 0.2916
Epoch 5/10
77/77 [==============================] - 0s 1ms/step - loss: 0.2892
Epoch 6/10
77/77 [==============================] - 0s 2ms/step - loss: 0.2868
Epoch 7/10
77/77 [==============================] - 0s 2ms/step - loss: 0.2860
Epoch 8/10
77/77 [==============================] - 0s 1ms/step - loss: 0.2863
Epoch 9/10
77/77 [==============================] - 0s 1ms/step - loss: 0.2865
Epoch 10/10
77/77 [==============================] - 0s 1ms/step - loss: 0.2861
Training loss: 0.526
Validation loss: 0.563


## Double input & bias

In [9]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(273*2,use_bias=True))
    model.add(layers.Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
kr = KerasRegressor(build_fn=build_model )

pipeline = Pipeline([
    ('transformer', transf),
    ('nn', kr)
])

pipeline.fit(df_train, y_train)


def rms(x):
    return np.sqrt(np.mean(x**2))
print('Training loss: %2.3f' % rms(y_train- pipeline.predict(df_train)))
print('Validation loss: %2.3f' % rms(y_val- pipeline.predict(df_val)))

C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/1217633080.py:7: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model )
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_2/dense_4/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_2/dense_4/embedding_lookup_sparse/Reshape:0", shape=(None, 546), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_2/dense_4/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


239/239 [==============================] - 1s 1ms/step - loss: 1.4035
Training loss: 0.542
Validation loss: 0.567


### 1 Extra layers

In [10]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(273*2,use_bias=True))
    model.add(layers.Dense(272*2,use_bias=False))
    model.add(layers.Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)

pipeline = Pipeline([
    ('transformer', transf),
    ('nn', kr)
])
pipeline.fit(df_train, y_train)

def rms(x):
    return np.sqrt(np.mean(x**2))
print('Training loss: %2.3f' % rms(y_train- pipeline.predict(df_train)))
print('Validation loss: %2.3f' % rms(y_val- pipeline.predict(df_val)))

Epoch 1/20


C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/1845786985.py:8: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_3/dense_6/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_3/dense_6/embedding_lookup_sparse/Reshape:0", shape=(None, 546), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_3/dense_6/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


8/8 [==============================] - 0s 20ms/step - loss: 7.2015
Epoch 2/20
8/8 [==============================] - 0s 22ms/step - loss: 1.6642
Epoch 3/20
8/8 [==============================] - 0s 22ms/step - loss: 0.9851
Epoch 4/20
8/8 [==============================] - 0s 26ms/step - loss: 0.5430
Epoch 5/20
8/8 [==============================] - 0s 28ms/step - loss: 0.3986
Epoch 6/20
8/8 [==============================] - 0s 21ms/step - loss: 0.3213
Epoch 7/20
8/8 [==============================] - 0s 22ms/step - loss: 0.2945
Epoch 8/20
8/8 [==============================] - 0s 47ms/step - loss: 0.2872
Epoch 9/20
8/8 [==============================] - 0s 28ms/step - loss: 0.2827
Epoch 10/20
8/8 [==============================] - 0s 30ms/step - loss: 0.2799
Epoch 11/20
8/8 [==============================] - 0s 24ms/step - loss: 0.2767
Epoch 12/20
8/8 [==============================] - 0s 26ms/step - loss: 0.2770
Epoch 13/20
8/8 [==============================] - 0s 31ms/step - loss: 

### with dropout

In [11]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(273,use_bias=True))
    model.add(layers.Dropout(.2))
    model.add(layers.Dense(272*2,use_bias=False))
    model.add(layers.Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)

pipeline = Pipeline([
    ('transformer', transf),
    ('nn', kr)
])
pipeline.fit(df_train, y_train)

def rms(x):
    return np.sqrt(np.mean(x**2))
print('Training loss: %2.3f' % rms(y_train- pipeline.predict(df_train)))
print('Validation loss: %2.3f' % rms(y_val- pipeline.predict(df_val)))

Epoch 1/20


C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/1518195368.py:9: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_4/dense_9/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_4/dense_9/embedding_lookup_sparse/Reshape:0", shape=(None, 273), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_4/dense_9/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


8/8 [==============================] - 1s 17ms/step - loss: 10.1109
Epoch 2/20
8/8 [==============================] - 0s 17ms/step - loss: 2.2800
Epoch 3/20
8/8 [==============================] - 0s 17ms/step - loss: 0.8676
Epoch 4/20
8/8 [==============================] - 0s 15ms/step - loss: 0.6779
Epoch 5/20
8/8 [==============================] - 0s 17ms/step - loss: 0.5526
Epoch 6/20
8/8 [==============================] - 0s 15ms/step - loss: 0.4589
Epoch 7/20
8/8 [==============================] - 0s 14ms/step - loss: 0.4115
Epoch 8/20
8/8 [==============================] - 0s 14ms/step - loss: 0.3983
Epoch 9/20
8/8 [==============================] - 0s 17ms/step - loss: 0.3905
Epoch 10/20
8/8 [==============================] - 0s 16ms/step - loss: 0.3777
Epoch 11/20
8/8 [==============================] - 0s 16ms/step - loss: 0.3758
Epoch 12/20
8/8 [==============================] - 0s 14ms/step - loss: 0.3789
Epoch 13/20
8/8 [==============================] - 0s 18ms/step - loss:

In [12]:
### Different dropout linear/dropout/linear/1

In [13]:
for dropout in np.linspace(0.1,0.9,10):

    def build_model():
        model = models.Sequential()
        model.add(layers.Dense(273,use_bias=True))
        model.add(layers.Dropout(dropout))
        model.add(layers.Dense(272,use_bias=False))
        model.add(layers.Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        return model
    kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)

    pipeline = Pipeline([
        ('transformer', transf),
        ('nn', kr)
    ])
    pipeline.fit(df_train, y_train)

    def rms(x):
        return np.sqrt(np.mean(x**2))
    print('Dropout: %2.3f' % dropout)
    print('-- Training loss: %2.3f' % rms(y_train- pipeline.predict(df_train)))
    print('-- Validation loss: %2.3f' % rms(y_val- pipeline.predict(df_val)))

Epoch 1/20


C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/2810878026.py:11: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_5/dense_12/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_5/dense_12/embedding_lookup_sparse/Reshape:0", shape=(None, 273), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_5/dense_12/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


8/8 [==============================] - 0s 12ms/step - loss: 10.8182
Epoch 2/20
8/8 [==============================] - 0s 12ms/step - loss: 2.0232
Epoch 3/20
8/8 [==============================] - 0s 14ms/step - loss: 0.9202
Epoch 4/20
8/8 [==============================] - 0s 13ms/step - loss: 0.7568
Epoch 5/20
8/8 [==============================] - 0s 12ms/step - loss: 0.4504
Epoch 6/20
8/8 [==============================] - 0s 14ms/step - loss: 0.3832
Epoch 7/20
8/8 [==============================] - 0s 15ms/step - loss: 0.3699
Epoch 8/20
8/8 [==============================] - 0s 13ms/step - loss: 0.3381
Epoch 9/20
8/8 [==============================] - 0s 15ms/step - loss: 0.3322
Epoch 10/20
8/8 [==============================] - 0s 15ms/step - loss: 0.3307
Epoch 11/20
8/8 [==============================] - 0s 14ms/step - loss: 0.3276
Epoch 12/20
8/8 [==============================] - 0s 12ms/step - loss: 0.3273
Epoch 13/20
8/8 [==============================] - 0s 12ms/step - loss:

C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/2810878026.py:11: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_6/dense_15/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_6/dense_15/embedding_lookup_sparse/Reshape:0", shape=(None, 273), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_6/dense_15/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


8/8 [==============================] - 0s 11ms/step - loss: 12.6416
Epoch 2/20
8/8 [==============================] - 0s 12ms/step - loss: 1.9020
Epoch 3/20
8/8 [==============================] - 0s 13ms/step - loss: 1.2059
Epoch 4/20
8/8 [==============================] - 0s 12ms/step - loss: 0.9280
Epoch 5/20
8/8 [==============================] - 0s 13ms/step - loss: 0.5136
Epoch 6/20
8/8 [==============================] - 0s 13ms/step - loss: 0.4612
Epoch 7/20
8/8 [==============================] - 0s 13ms/step - loss: 0.4236
Epoch 8/20
8/8 [==============================] - 0s 13ms/step - loss: 0.3871
Epoch 9/20
8/8 [==============================] - 0s 13ms/step - loss: 0.3742
Epoch 10/20
8/8 [==============================] - 0s 12ms/step - loss: 0.3822
Epoch 11/20
8/8 [==============================] - 0s 12ms/step - loss: 0.3664
Epoch 12/20
8/8 [==============================] - 0s 13ms/step - loss: 0.3715
Epoch 13/20
8/8 [==============================] - 0s 16ms/step - loss:

...

8/8 [==============================] - 0s 14ms/step - loss: 12.7318
Epoch 2/20
8/8 [==============================] - 0s 15ms/step - loss: 2.7571
Epoch 3/20
8/8 [==============================] - 0s 17ms/step - loss: 1.9754
Epoch 4/20
8/8 [==============================] - 0s 16ms/step - loss: 1.6920
Epoch 5/20
8/8 [==============================] - 0s 15ms/step - loss: 1.2406
Epoch 6/20
8/8 [==============================] - 0s 15ms/step - loss: 1.1553
Epoch 7/20
8/8 [==============================] - 0s 13ms/step - loss: 1.0800
Epoch 8/20
8/8 [==============================] - 0s 14ms/step - loss: 0.9629
Epoch 9/20
8/8 [==============================] - 0s 14ms/step - loss: 0.9609
Epoch 10/20
8/8 [==============================] - 0s 14ms/step - loss: 0.8975
Epoch 11/20
8/8 [==============================] - 0s 14ms/step - loss: 0.8630
Epoch 12/20
8/8 [==============================] - 0s 16ms/step - loss: 0.8329
Epoch 13/20
8/8 [==============================] - 0s 14ms/step - loss:

C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/2810878026.py:11: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_13/dense_36/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_13/dense_36/embedding_lookup_sparse/Reshape:0", shape=(None, 273), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_13/dense_36/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


8/8 [==============================] - 0s 12ms/step - loss: 12.0295
Epoch 2/20
8/8 [==============================] - 0s 14ms/step - loss: 3.1503
Epoch 3/20
8/8 [==============================] - 0s 14ms/step - loss: 2.4273
Epoch 4/20
8/8 [==============================] - 0s 18ms/step - loss: 2.1890
Epoch 5/20
8/8 [==============================] - 0s 14ms/step - loss: 1.6310
Epoch 6/20
8/8 [==============================] - 0s 17ms/step - loss: 1.4526
Epoch 7/20
8/8 [==============================] - 0s 28ms/step - loss: 1.3656
Epoch 8/20
8/8 [==============================] - 0s 16ms/step - loss: 1.2908
Epoch 9/20
8/8 [==============================] - 0s 17ms/step - loss: 1.2072
Epoch 10/20
8/8 [==============================] - 0s 17ms/step - loss: 1.1374
Epoch 11/20
8/8 [==============================] - 0s 16ms/step - loss: 1.1240
Epoch 12/20
8/8 [==============================] - 0s 17ms/step - loss: 1.0596
Epoch 13/20
8/8 [==============================] - 0s 13ms/step - loss:

C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/2810878026.py:11: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_14/dense_39/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_14/dense_39/embedding_lookup_sparse/Reshape:0", shape=(None, 273), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_14/dense_39/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


8/8 [==============================] - 1s 20ms/step - loss: 12.6470
Epoch 2/20
8/8 [==============================] - 0s 19ms/step - loss: 4.6371
Epoch 3/20
8/8 [==============================] - 0s 17ms/step - loss: 3.6250
Epoch 4/20
8/8 [==============================] - 0s 17ms/step - loss: 3.1007
Epoch 5/20
8/8 [==============================] - 0s 14ms/step - loss: 2.4176
Epoch 6/20
8/8 [==============================] - 0s 16ms/step - loss: 2.1899
Epoch 7/20
8/8 [==============================] - 0s 15ms/step - loss: 2.0201
Epoch 8/20
8/8 [==============================] - 0s 17ms/step - loss: 1.8786
Epoch 9/20
8/8 [==============================] - 0s 15ms/step - loss: 1.7601
Epoch 10/20
8/8 [==============================] - 0s 15ms/step - loss: 1.6811
Epoch 11/20
8/8 [==============================] - 0s 16ms/step - loss: 1.6449
Epoch 12/20
8/8 [==============================] - 0s 15ms/step - loss: 1.5666
Epoch 13/20
8/8 [==============================] - 0s 12ms/step - loss:

### linear/dropout/1

In [14]:
for dropout in np.linspace(0.1,0.5,10):

    def build_model():
        model = models.Sequential()
        model.add(layers.Dense(273,use_bias=True))
        model.add(layers.Dropout(dropout))
        model.add(layers.Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        return model
    kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)

    pipeline = Pipeline([
        ('transformer', transf),
        ('nn', kr)
    ])
    pipeline.fit(df_train, y_train)

    def rms(x):
        return np.sqrt(np.mean(x**2))
    print('Dropout: %2.3f' % dropout)
    print('-- Training loss: %2.3f' % rms(y_train- pipeline.predict(df_train)))
    print('-- Validation loss: %2.3f' % rms(y_val- pipeline.predict(df_val)))

Epoch 1/20


C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/4259746530.py:10: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_15/dense_42/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_15/dense_42/embedding_lookup_sparse/Reshape:0", shape=(None, 273), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_15/dense_42/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


8/8 [==============================] - 1s 11ms/step - loss: 17.7313
Epoch 2/20
8/8 [==============================] - 0s 11ms/step - loss: 10.9926
Epoch 3/20
8/8 [==============================] - 0s 9ms/step - loss: 5.9841
Epoch 4/20
8/8 [==============================] - 0s 10ms/step - loss: 2.8772
Epoch 5/20
8/8 [==============================] - 0s 11ms/step - loss: 1.6529
Epoch 6/20
8/8 [==============================] - 0s 11ms/step - loss: 1.3754
Epoch 7/20
8/8 [==============================] - 0s 10ms/step - loss: 1.1245
Epoch 8/20
8/8 [==============================] - 0s 10ms/step - loss: 0.8514
Epoch 9/20
8/8 [==============================] - 0s 9ms/step - loss: 0.6804
Epoch 10/20
8/8 [==============================] - 0s 13ms/step - loss: 0.5774
Epoch 11/20
8/8 [==============================] - 0s 13ms/step - loss: 0.4978
Epoch 12/20
8/8 [==============================] - 0s 9ms/step - loss: 0.4400
Epoch 13/20
8/8 [==============================] - 0s 11ms/step - loss: 0

C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/4259746530.py:10: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_16/dense_44/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_16/dense_44/embedding_lookup_sparse/Reshape:0", shape=(None, 273), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_16/dense_44/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


8/8 [==============================] - 1s 10ms/step - loss: 16.9600
Epoch 2/20
8/8 [==============================] - 0s 12ms/step - loss: 10.1329
Epoch 3/20
8/8 [==============================] - 0s 10ms/step - loss: 5.1330
Epoch 4/20
8/8 [==============================] - 0s 13ms/step - loss: 2.2806
Epoch 5/20
8/8 [==============================] - 0s 22ms/step - loss: 1.3486
Epoch 6/20
8/8 [==============================] - 0s 18ms/step - loss: 1.2209
Epoch 7/20
8/8 [==============================] - 0s 14ms/step - loss: 0.9702
Epoch 8/20
8/8 [==============================] - 0s 15ms/step - loss: 0.7276
Epoch 9/20
8/8 [==============================] - 0s 11ms/step - loss: 0.6064
Epoch 10/20
8/8 [==============================] - 0s 11ms/step - loss: 0.5267
Epoch 11/20
8/8 [==============================] - 0s 9ms/step - loss: 0.4618
Epoch 12/20
8/8 [==============================] - 0s 10ms/step - loss: 0.4124
Epoch 13/20
8/8 [==============================] - 0s 16ms/step - loss:

...

8/8 [==============================] - 0s 10ms/step - loss: 17.2409
Epoch 2/20
8/8 [==============================] - 0s 11ms/step - loss: 10.2535
Epoch 3/20
8/8 [==============================] - 0s 12ms/step - loss: 5.2689
Epoch 4/20
8/8 [==============================] - 0s 12ms/step - loss: 2.5079
Epoch 5/20
8/8 [==============================] - 0s 11ms/step - loss: 1.6819
Epoch 6/20
8/8 [==============================] - 0s 12ms/step - loss: 1.4814
Epoch 7/20
8/8 [==============================] - 0s 12ms/step - loss: 1.1883
Epoch 8/20
8/8 [==============================] - 0s 11ms/step - loss: 0.9128
Epoch 9/20
8/8 [==============================] - 0s 11ms/step - loss: 0.7387
Epoch 10/20
8/8 [==============================] - 0s 11ms/step - loss: 0.6613
Epoch 11/20
8/8 [==============================] - 0s 11ms/step - loss: 0.5754
Epoch 12/20
8/8 [==============================] - 0s 20ms/step - loss: 0.5077
Epoch 13/20
8/8 [==============================] - 0s 16ms/step - loss

C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/4259746530.py:10: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_23/dense_58/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_23/dense_58/embedding_lookup_sparse/Reshape:0", shape=(None, 273), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_23/dense_58/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


8/8 [==============================] - 0s 11ms/step - loss: 18.2257
Epoch 2/20
8/8 [==============================] - 0s 11ms/step - loss: 11.1306
Epoch 3/20
8/8 [==============================] - 0s 12ms/step - loss: 5.9112
Epoch 4/20
8/8 [==============================] - 0s 12ms/step - loss: 2.7587
Epoch 5/20
8/8 [==============================] - 0s 10ms/step - loss: 1.6005
Epoch 6/20
8/8 [==============================] - 0s 13ms/step - loss: 1.3830
Epoch 7/20
8/8 [==============================] - 0s 11ms/step - loss: 1.1779
Epoch 8/20
8/8 [==============================] - 0s 11ms/step - loss: 0.9036
Epoch 9/20
8/8 [==============================] - 0s 10ms/step - loss: 0.7511
Epoch 10/20
8/8 [==============================] - 0s 11ms/step - loss: 0.6395
Epoch 11/20
8/8 [==============================] - 0s 12ms/step - loss: 0.5899
Epoch 12/20
8/8 [==============================] - 0s 11ms/step - loss: 0.5437
Epoch 13/20
8/8 [==============================] - 0s 11ms/step - loss

C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/4259746530.py:10: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_24/dense_60/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_24/dense_60/embedding_lookup_sparse/Reshape:0", shape=(None, 273), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_24/dense_60/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


8/8 [==============================] - 1s 10ms/step - loss: 17.9677
Epoch 2/20
8/8 [==============================] - 0s 9ms/step - loss: 11.2851
Epoch 3/20
8/8 [==============================] - 0s 10ms/step - loss: 6.2278
Epoch 4/20
8/8 [==============================] - 0s 10ms/step - loss: 3.1133
Epoch 5/20
8/8 [==============================] - 0s 11ms/step - loss: 1.8194
Epoch 6/20
8/8 [==============================] - 0s 12ms/step - loss: 1.5280
Epoch 7/20
8/8 [==============================] - 0s 12ms/step - loss: 1.3145
Epoch 8/20
8/8 [==============================] - 0s 15ms/step - loss: 0.9884
Epoch 9/20
8/8 [==============================] - 0s 10ms/step - loss: 0.8266
Epoch 10/20
8/8 [==============================] - 0s 10ms/step - loss: 0.7294
Epoch 11/20
8/8 [==============================] - 0s 10ms/step - loss: 0.6450
Epoch 12/20
8/8 [==============================] - 0s 10ms/step - loss: 0.5813
Epoch 13/20
8/8 [==============================] - 0s 9ms/step - loss: 

### with dropout, 3 inner layers

In [15]:
for dropout in np.linspace(0.1,0.5,5):

    def build_model():
        model = models.Sequential()
        model.add(layers.Dense(273,use_bias=True))
        model.add(layers.Dropout(dropout))
        model.add(layers.Dense(136,use_bias=True))
        model.add(layers.Dense(68,use_bias=True))
        model.add(layers.Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        return model
    kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)

    pipeline = Pipeline([
        ('transformer', transf),
        ('nn', kr)
    ])
    pipeline.fit(df_train, y_train)

    def rms(x):
        return np.sqrt(np.mean(x**2))
    print('Dropout: %2.3f' % dropout)
    print('-- Training loss: %2.3f' % rms(y_train- pipeline.predict(df_train)))
    print('-- Validation loss: %2.3f' % rms(y_val- pipeline.predict(df_val)))

Epoch 1/20


C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/4248957711.py:12: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_25/dense_62/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_25/dense_62/embedding_lookup_sparse/Reshape:0", shape=(None, 273), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_25/dense_62/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


8/8 [==============================] - 1s 14ms/step - loss: 11.2854
Epoch 2/20
8/8 [==============================] - 0s 11ms/step - loss: 2.2111
Epoch 3/20
8/8 [==============================] - 0s 12ms/step - loss: 0.8899
Epoch 4/20
8/8 [==============================] - 0s 14ms/step - loss: 0.5952
Epoch 5/20
8/8 [==============================] - 0s 12ms/step - loss: 0.4957
Epoch 6/20
8/8 [==============================] - 0s 15ms/step - loss: 0.4074
Epoch 7/20
8/8 [==============================] - 0s 13ms/step - loss: 0.3583
Epoch 8/20
8/8 [==============================] - 0s 15ms/step - loss: 0.3357
Epoch 9/20
8/8 [==============================] - 0s 12ms/step - loss: 0.3301
Epoch 10/20
8/8 [==============================] - 0s 13ms/step - loss: 0.3263
Epoch 11/20
8/8 [==============================] - 0s 12ms/step - loss: 0.3198
Epoch 12/20
8/8 [==============================] - 0s 13ms/step - loss: 0.3207
Epoch 13/20
8/8 [==============================] - 0s 15ms/step - loss:

C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/4248957711.py:12: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_26/dense_66/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_26/dense_66/embedding_lookup_sparse/Reshape:0", shape=(None, 273), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_26/dense_66/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


8/8 [==============================] - 0s 14ms/step - loss: 11.5121
Epoch 2/20
8/8 [==============================] - 0s 14ms/step - loss: 2.0781
Epoch 3/20
8/8 [==============================] - 0s 17ms/step - loss: 0.9179
Epoch 4/20
8/8 [==============================] - 0s 16ms/step - loss: 0.6992
Epoch 5/20
8/8 [==============================] - 0s 16ms/step - loss: 0.5299
Epoch 6/20
8/8 [==============================] - 0s 14ms/step - loss: 0.4387
Epoch 7/20
8/8 [==============================] - 0s 14ms/step - loss: 0.4137
Epoch 8/20
8/8 [==============================] - 0s 14ms/step - loss: 0.3915
Epoch 9/20
8/8 [==============================] - 0s 16ms/step - loss: 0.3630
Epoch 10/20
8/8 [==============================] - 0s 16ms/step - loss: 0.3613
Epoch 11/20
8/8 [==============================] - 0s 14ms/step - loss: 0.3591
Epoch 12/20
8/8 [==============================] - 0s 16ms/step - loss: 0.3621
Epoch 13/20
8/8 [==============================] - 0s 18ms/step - loss:

...

8/8 [==============================] - 0s 12ms/step - loss: 11.2263
Epoch 2/20
8/8 [==============================] - 0s 13ms/step - loss: 2.1243
Epoch 3/20
8/8 [==============================] - 0s 14ms/step - loss: 0.9512
Epoch 4/20
8/8 [==============================] - 0s 14ms/step - loss: 0.7274
Epoch 5/20
8/8 [==============================] - 0s 18ms/step - loss: 0.5897
Epoch 6/20
8/8 [==============================] - 0s 25ms/step - loss: 0.4872
Epoch 7/20
8/8 [==============================] - 0s 14ms/step - loss: 0.4504
Epoch 8/20
8/8 [==============================] - 0s 14ms/step - loss: 0.4464
Epoch 9/20
8/8 [==============================] - 0s 13ms/step - loss: 0.4218
Epoch 10/20
8/8 [==============================] - 0s 12ms/step - loss: 0.4211
Epoch 11/20
8/8 [==============================] - 0s 13ms/step - loss: 0.4137
Epoch 12/20
8/8 [==============================] - 0s 14ms/step - loss: 0.4064
Epoch 13/20
8/8 [==============================] - 0s 16ms/step - loss:

C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/4248957711.py:12: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_28/dense_74/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_28/dense_74/embedding_lookup_sparse/Reshape:0", shape=(None, 273), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_28/dense_74/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


8/8 [==============================] - 1s 11ms/step - loss: 10.6612
Epoch 2/20
8/8 [==============================] - 0s 12ms/step - loss: 2.2740
Epoch 3/20
8/8 [==============================] - 0s 14ms/step - loss: 1.0496
Epoch 4/20
8/8 [==============================] - 0s 13ms/step - loss: 0.7884
Epoch 5/20
8/8 [==============================] - 0s 12ms/step - loss: 0.6845
Epoch 6/20
8/8 [==============================] - 0s 12ms/step - loss: 0.5753
Epoch 7/20
8/8 [==============================] - 0s 12ms/step - loss: 0.5484
Epoch 8/20
8/8 [==============================] - 0s 17ms/step - loss: 0.5099
Epoch 9/20
8/8 [==============================] - 0s 15ms/step - loss: 0.5033
Epoch 10/20
8/8 [==============================] - 0s 13ms/step - loss: 0.5000
Epoch 11/20
8/8 [==============================] - 0s 12ms/step - loss: 0.4819
Epoch 12/20
8/8 [==============================] - 0s 16ms/step - loss: 0.4756
Epoch 13/20
8/8 [==============================] - 0s 24ms/step - loss:

C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/4248957711.py:12: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=20,batch_size=1000)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_29/dense_78/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_29/dense_78/embedding_lookup_sparse/Reshape:0", shape=(None, 273), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_29/dense_78/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


8/8 [==============================] - 0s 13ms/step - loss: 12.2982
Epoch 2/20
8/8 [==============================] - 0s 15ms/step - loss: 2.4713
Epoch 3/20
8/8 [==============================] - 0s 12ms/step - loss: 1.2100
Epoch 4/20
8/8 [==============================] - 0s 12ms/step - loss: 1.0240
Epoch 5/20
8/8 [==============================] - 0s 12ms/step - loss: 0.8235
Epoch 6/20
8/8 [==============================] - 0s 13ms/step - loss: 0.6800
Epoch 7/20
8/8 [==============================] - 0s 14ms/step - loss: 0.6597
Epoch 8/20
8/8 [==============================] - 0s 13ms/step - loss: 0.5929
Epoch 9/20
8/8 [==============================] - 0s 14ms/step - loss: 0.5884
Epoch 10/20
8/8 [==============================] - 0s 13ms/step - loss: 0.5707
Epoch 11/20
8/8 [==============================] - 0s 13ms/step - loss: 0.5505
Epoch 12/20
8/8 [==============================] - 0s 12ms/step - loss: 0.5384
Epoch 13/20
8/8 [==============================] - 0s 11ms/step - loss:

### Random uniform

In [16]:
def build_model():
    model = models.Sequential()
    initializer = tf.keras.initializers.RandomUniform(minval=-1., maxval=1.)
    model.add(layers.Dense(273,use_bias=True, kernel_initializer=initializer))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
kr = KerasRegressor(build_fn=build_model,epochs=100,batch_size=100)

pipeline = Pipeline([
    ('transformer', transf),
    ('nn', kr)
])
pipeline.fit(df_train, y_train)

def rms(x):
    return np.sqrt(np.mean(x**2))
print('Dropout: %2.3f' % dropout)
print('-- Training loss: %2.3f' % rms(y_train- pipeline.predict(df_train)))
print('-- Validation loss: %2.3f' % rms(y_val- pipeline.predict(df_val)))

Epoch 1/100


C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/1329411624.py:9: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=100,batch_size=100)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_30/dense_82/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_30/dense_82/embedding_lookup_sparse/Reshape:0", shape=(None, 273), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_30/dense_82/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


77/77 [==============================] - 1s 3ms/step - loss: 4.1793
Epoch 2/100
77/77 [==============================] - 1s 6ms/step - loss: 0.8804
Epoch 3/100
77/77 [==============================] - 0s 5ms/step - loss: 0.7121
Epoch 4/100
77/77 [==============================] - 0s 4ms/step - loss: 0.6719
Epoch 5/100
77/77 [==============================] - 0s 4ms/step - loss: 0.6247
Epoch 6/100
77/77 [==============================] - 0s 3ms/step - loss: 0.5912
Epoch 7/100
77/77 [==============================] - 0s 3ms/step - loss: 0.5749
Epoch 8/100
77/77 [==============================] - 0s 3ms/step - loss: 0.5754
Epoch 9/100
77/77 [==============================] - 0s 3ms/step - loss: 0.5320
Epoch 10/100
77/77 [==============================] - 0s 3ms/step - loss: 0.5023
Epoch 11/100
77/77 [==============================] - 0s 3ms/step - loss: 0.4889
Epoch 12/100
77/77 [==============================] - 0s 3ms/step - loss: 0.4545
Epoch 13/100
77/77 [=============================

#### 1-layer

In [17]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(1,use_bias=True))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100)

pipeline = Pipeline([
    ('transformer', transf),
    ('nn', kr)
])
pipeline.fit(df_train, y_train)

def rms(x):
    return np.sqrt(np.mean(x**2))
print('-- Training loss: %2.3f' % rms(y_train- pipeline.predict(df_train)))
print('-- Validation loss: %2.3f' % rms(y_val- pipeline.predict(df_val)))

Epoch 1/150


C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/4205123862.py:6: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_31/dense_84/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_31/dense_84/embedding_lookup_sparse/Reshape:0", shape=(None, 1), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_31/dense_84/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


77/77 [==============================] - 1s 1ms/step - loss: 19.6442
Epoch 2/150
77/77 [==============================] - 0s 1ms/step - loss: 16.6070
Epoch 3/150
77/77 [==============================] - 0s 1ms/step - loss: 13.9687
Epoch 4/150
77/77 [==============================] - 0s 988us/step - loss: 11.6995
Epoch 5/150
77/77 [==============================] - 0s 1ms/step - loss: 9.7638
Epoch 6/150
77/77 [==============================] - 0s 1ms/step - loss: 8.1270
Epoch 7/150
77/77 [==============================] - 0s 1ms/step - loss: 6.7538
Epoch 8/150
77/77 [==============================] - 0s 1ms/step - loss: 5.6118
Epoch 9/150
77/77 [==============================] - 0s 3ms/step - loss: 4.6725
Epoch 10/150
77/77 [==============================] - 0s 1ms/step - loss: 3.9053
Epoch 11/150
77/77 [==============================] - 0s 1ms/step - loss: 3.2829
Epoch 12/150
77/77 [==============================] - 0s 1ms/step - loss: 2.7826
Epoch 13/150
77/77 [=======================

# Different optimizer

In [18]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(1,use_bias=True))
    model.compile(loss='mean_squared_error', optimizer='adamax')
    return model
kr = KerasRegressor(build_fn=build_model,epochs=200,batch_size=100)

pipeline = Pipeline([
    ('transformer', transf),
    ('nn', kr)
])
pipeline.fit(df_train, y_train)

def rms(x):
    return np.sqrt(np.mean(x**2))
print('-- Training loss: %2.3f' % rms(y_train- pipeline.predict(df_train)))
print('-- Validation loss: %2.3f' % rms(y_val- pipeline.predict(df_val)))

Epoch 1/200


C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/1292449798.py:6: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=200,batch_size=100)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_32/dense_85/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_32/dense_85/embedding_lookup_sparse/Reshape:0", shape=(None, 1), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_32/dense_85/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


77/77 [==============================] - 0s 929us/step - loss: 19.5303
Epoch 2/200
77/77 [==============================] - 0s 1ms/step - loss: 17.3175
Epoch 3/200
77/77 [==============================] - 0s 983us/step - loss: 15.2993
Epoch 4/200
77/77 [==============================] - 0s 929us/step - loss: 13.4647
Epoch 5/200
77/77 [==============================] - 0s 968us/step - loss: 11.8153
Epoch 6/200
77/77 [==============================] - 0s 996us/step - loss: 10.3178
Epoch 7/200
77/77 [==============================] - 0s 1ms/step - loss: 8.9647
Epoch 8/200
77/77 [==============================] - 0s 1ms/step - loss: 7.7444
Epoch 9/200
77/77 [==============================] - 0s 961us/step - loss: 6.6579
Epoch 10/200
77/77 [==============================] - 0s 1ms/step - loss: 5.7006
Epoch 11/200
77/77 [==============================] - 0s 996us/step - loss: 4.8678
Epoch 12/200
77/77 [==============================] - 0s 1ms/step - loss: 4.1579
Epoch 13/200
77/77 [=========

# adamax with different learning rates

In [19]:
for lr in [1e-4,5e-4,1e-3,2e-3,5e-3,1e-2]:
    def build_model():
        model = models.Sequential()
        model.add(layers.Dense(1,use_bias=True))
        opt = keras.optimizers.Adamax(learning_rate = lr)
        model.compile(loss='mean_squared_error', optimizer=opt)
        return model
    kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)

    pipeline = Pipeline([
        ('transformer', transf),
        ('nn', kr)
    ])
    pipeline.fit(df_train, y_train)

    def rms(x):
        return np.sqrt(np.mean(x**2))
    print('lr: %2.3e' % lr)
    print('-- Training loss: %2.3f' % rms(y_train- pipeline.predict(df_train)))
    print('-- Validation loss: %2.3f' % rms(y_val- pipeline.predict(df_val)))

C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/1552145588.py:8: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_33/dense_86/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_33/dense_86/embedding_lookup_sparse/Reshape:0", shape=(None, 1), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_33/dense_86/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


lr: 1.000e-04
-- Training loss: 1.442
-- Validation loss: 1.425


C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/1552145588.py:8: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_34/dense_87/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_34/dense_87/embedding_lookup_sparse/Reshape:0", shape=(None, 1), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_34/dense_87/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


lr: 5.000e-04
-- Training loss: 0.541
-- Validation loss: 0.554


C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/1552145588.py:8: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_35/dense_88/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_35/dense_88/embedding_lookup_sparse/Reshape:0", shape=(None, 1), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_35/dense_88/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


...

lr: 2.000e-03
-- Training loss: 0.531
-- Validation loss: 0.553


C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/1552145588.py:8: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)


lr: 5.000e-03
-- Training loss: 0.528
-- Validation loss: 0.555


C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_37/dense_90/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_37/dense_90/embedding_lookup_sparse/Reshape:0", shape=(None, 1), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_37/dense_90/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/1552145588.py:8: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)
C:\Users\kay_t\AppData\Local\Package

lr: 1.000e-02
-- Training loss: 0.525
-- Validation loss: 0.558


## adam with different learning rates

In [20]:
for lr in [1e-4,5e-4,1e-3,2e-3,5e-3,1e-2]:
    def build_model():
        model = models.Sequential()
        model.add(layers.Dense(1,use_bias=True))
        opt = keras.optimizers.Adam(learning_rate = lr)
        model.compile(loss='mean_squared_error', optimizer=opt)
        return model
    kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)

    pipeline = Pipeline([
        ('transformer', transf),
        ('nn', kr)
    ])
    pipeline.fit(df_train, y_train)

    def rms(x):
        return np.sqrt(np.mean(x**2))
    print('lr: %2.3e' % lr)
    print('-- Training loss: %2.3f' % rms(y_train- pipeline.predict(df_train)))
    print('-- Validation loss: %2.3f' % rms(y_val- pipeline.predict(df_val)))

C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/1025543893.py:8: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_39/dense_92/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_39/dense_92/embedding_lookup_sparse/Reshape:0", shape=(None, 1), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_39/dense_92/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


lr: 1.000e-04
-- Training loss: 0.989
-- Validation loss: 0.982


C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/1025543893.py:8: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_40/dense_93/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_40/dense_93/embedding_lookup_sparse/Reshape:0", shape=(None, 1), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_40/dense_93/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


lr: 5.000e-04
-- Training loss: 0.528
-- Validation loss: 0.556


C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/1025543893.py:8: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_41/dense_94/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_41/dense_94/embedding_lookup_sparse/Reshape:0", shape=(None, 1), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_41/dense_94/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


...

lr: 2.000e-03
-- Training loss: 0.523
-- Validation loss: 0.561


C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/1025543893.py:8: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_43/dense_96/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_43/dense_96/embedding_lookup_sparse/Reshape:0", shape=(None, 1), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_43/dense_96/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


lr: 5.000e-03
-- Training loss: 0.525
-- Validation loss: 0.563


C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/1025543893.py:8: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_44/dense_97/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_44/dense_97/embedding_lookup_sparse/Reshape:0", shape=(None, 1), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_44/dense_97/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


lr: 1.000e-02
-- Training loss: 0.524
-- Validation loss: 0.562


### RMSProp

In [22]:

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(1,use_bias=True))
    opt = keras.optimizers.RMSprop()
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model
kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)

pipeline = Pipeline([
    ('transformer', transf),
    ('nn', kr)
])
pipeline.fit(df_train, y_train)

def rms(x):
    return np.sqrt(np.mean(x**2))
print('lr: %2.3e' % lr)
print('-- Training loss: %2.3f' % rms(y_train- pipeline.predict(df_train)))
print('-- Validation loss: %2.3f' % rms(y_val- pipeline.predict(df_val)))



C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/112345705.py:7: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_46/dense_99/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_46/dense_99/embedding_lookup_sparse/Reshape:0", shape=(None, 1), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_46/dense_99/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


lr: 1.000e-02
-- Training loss: 0.528
-- Validation loss: 0.553


### trying with different optimizers

In [27]:

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(1,use_bias=False))
    #opt = keras.optimizers.RMSprop()
    #opt = keras.optimizers.Adagrad()
    opt = keras.optimizers.SGD()
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model
kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)

pipeline = Pipeline([
    ('transformer', transf),
    ('nn', kr)
])
pipeline.fit(df_train, y_train)

def rms(x):
    return np.sqrt(np.mean(x**2))
print('lr: %2.3e' % lr)
print('-- Training loss: %2.3f' % rms(y_train- pipeline.predict(df_train)))
print('-- Validation loss: %2.3f' % rms(y_val- pipeline.predict(df_val)))



C:\Users\kay_t\AppData\Local\Temp/ipykernel_22192/80736350.py:9: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  kr = KerasRegressor(build_fn=build_model,epochs=150,batch_size=100,verbose=False)
C:\Users\kay_t\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_51/dense_106/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_51/dense_106/embedding_lookup_sparse/Reshape:0", shape=(None, 1), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_51/dense_106/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


lr: 1.000e-02
-- Training loss: 0.535
-- Validation loss: 0.552
